# To Do:

1. Given neural net, form the NTK and do gradient descent on the kernel regression.
2. Consolidate the resampling of the data, the forming of the neural net and NTK, and training into same class.
3. Perform the test that we have multiple times (~100 times) to get a smoothed version of the permutation test.

# Overview

Given two datasets $A$ and $B$, which come respectively from distributions $P$ and $Q$, two-sample tests are basically a statistical test that tries to answer the hypothesis $H_0: P = Q$ (or reject it in favor of $H_1: P \neq Q$).

The goal of this notebook is to investigate the phenomena of the neural net two-sample test, which basically uses the time to learn the two distributions as the statistical test.  So what we will do is

1.  Create a hard two sample testing problem by creating $A$ and $B$ from some $P$ and $Q$ that are very hard to differentiate.
2.  Create a base neural net class (with last layer as either symmetrized initialization, 0 initialization, or random).  In each case, we need to run a permutation test and plot figures of the data with the color map of the outputs and witness function function values.



In [ ]:
!pip uninstall tensorflow
pip uninstall tensorflow-probability

In [ ]:
# import statements
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.data import Dataset
from tensorflow import keras
from keras import layers
import pandas as pd
import random
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tqdm import trange
import multiprocessing
from multiprocessing import Pool, Manager
import datasets
import gc
from IPython.display import clear_output
import numpy as np
import time
import math
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation as LatentDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LinearDA

# Data

We will let $P$ and $Q$ both be Gaussian mixture models given by

$$P = \sum_{i=1}^2 \frac{1}{2} \mathcal{N}( \mu_i^h , I_d )$$
$$Q = \sum_{i=1}^2 \frac{1}{2} \mathcal{N}\Bigg( \mu_i^h , \begin{bmatrix}
    1 & \Delta_i^h & 0_{d-2} \\ \Delta_i^h & 1 & 0_{d-2} \\ 0_{d-2}^\top & 0_{d-2}^\top & I_{d-2}
\end{bmatrix} \Bigg),$$
where $\mu_1^h = 0_d$, $\mu_2^h = 0.5 * \mathbf{1}_d$, $\Delta_1^h = 0.5$, and $\Delta_2^h = -0.5$.

In [ ]:
class Data_Constructor(object):
    def __init__(self, trainN=5000, testN=1000, d=20, batch_sz=32):
        # set hyperparams
        self.trainN = trainN
        self.testN = testN
        self.N = self.trainN + self.testN
        self.d = d
        self.batch_sz = batch_sz
        
        # mean vectors
        self.mu_1 = tf.zeros(self.d)
        self.mu_2 = 0.5*tf.ones(d)
        # covariance offsets
        delta_1 = 0.5
        delta_2 = -0.5
        # define covariances
        self.I = tf.eye(self.d)
        self.sig_1 = tf.linalg.LinearOperatorBlockDiag([
                        tf.linalg.LinearOperatorFullMatrix(tf.constant([ [1, delta_1], [delta_1, 1] ])),
                        tf.linalg.LinearOperatorFullMatrix(tf.eye(d-2))])
        self.sig_1 = self.sig_1.to_dense()
        self.sig_2 = tf.linalg.LinearOperatorBlockDiag([
                        tf.linalg.LinearOperatorFullMatrix(tf.constant([ [1, delta_2], [delta_2, 1] ])),
                        tf.linalg.LinearOperatorFullMatrix(tf.eye(d-2))])
        self.sig_2 = self.sig_2.to_dense()
        # categorical sampling methods
        self.bernoulli_dist = tfp.distributions.Bernoulli(probs=[0.5])
        # construct gaussian distributions
        self.gauss_mix_1 = [tfp.distributions.MultivariateNormalFullCovariance(
                            loc=self.mu_1, covariance_matrix=self.I), 
                       tfp.distributions.MultivariateNormalFullCovariance(
                            loc=self.mu_2, covariance_matrix=self.I)]
        self.gauss_mix_2 = [tfp.distributions.MultivariateNormalFullCovariance(
                            loc=self.mu_1, covariance_matrix=self.sig_1), 
                       tfp.distributions.MultivariateNormalFullCovariance(
                            loc=self.mu_2, covariance_matrix=self.sig_2)]

    def constructData(self):
        # figure out how many to sample from first Gaussian dist vs second Gaussian dist
        cat_dist_P = self.bernoulli_dist.sample(self.N)
        cat_dist_Q = self.bernoulli_dist.sample(self.N)
        
        samp_dist_P = tf.math.reduce_sum(cat_dist_P)
        dset_A_X = tf.concat([self.gauss_mix_1[0].sample(samp_dist_P),
                              self.gauss_mix_1[1].sample(self.N-samp_dist_P)],
                                 axis=0)
        dset_A_Y = tf.expand_dims(tf.zeros(self.N),axis=1)
        dset_A_X_prime = self.gauss_mix_1[0].sample(self.N)
        
        samp_dist_Q = tf.math.reduce_sum(cat_dist_Q)
        dset_B_X = tf.concat([self.gauss_mix_2[0].sample(samp_dist_Q),
                             self.gauss_mix_2[1].sample(self.N-samp_dist_Q)],
                                 axis=0)
        dset_B_Y = tf.expand_dims(tf.ones(self.N), axis=1)
        dset_B_X_prime = self.gauss_mix_2[0].sample(self.N)
        # aggregate data from A and B
        dset_X = tf.concat([dset_A_X, dset_B_X], axis=0)
        dset_Y = tf.concat([dset_A_Y, dset_B_Y], axis=0)

        # prime dataset
        dset_X_prime = tf.concat([dset_A_X_prime, dset_B_X_prime], axis=0)
        dset_Y_prime = tf.concat([dset_A_Y, dset_B_Y], axis=0)

        # actually make dataset
        dset = tf.data.Dataset.from_tensor_slices((dset_X,
                        dset_Y)).shuffle(2*self.N).shuffle(2*self.N)
        dset_Train = dset.take(self.trainN*2)
        dset_Train = dset_Train.batch(self.batch_sz)

        dset_Test = dset.skip(self.trainN*2)
        dset_Test = dset_Test.batch(self.batch_sz)

        # make prime dataset
        dset_prime = tf.data.Dataset.from_tensor_slices((dset_X_prime,
                        dset_Y_prime)).shuffle(2*self.N).shuffle(2*self.N)
        dset_Train_prime = dset_prime.take(self.trainN*2)
        dset_Train_prime = dset_Train_prime.batch(self.batch_sz)

        dset_Test_prime = dset_prime.skip(self.trainN*2)
        dset_Test_prime = dset_Test_prime.batch(self.batch_sz)
        return dset_Train, dset_Test, dset_Train_prime, dset_Test_prime
    



# Neural Network

We will create a neural network that is able to be initialized randomly, from zero, and in a symmetric manner.

In [ ]:
class symmetricLastLayer_Init(tf.keras.initializers.Initializer):
    def __init__(self, ):
        pass
    
    def __call__(self, shape, dtype=tf.float32, **kwargs):
        input_shape = shape[0]
        if input_shape % 2 == 0: # even input shape
            init_weights = tf.concat( [tf.ones(input_shape//2,dtype=dtype),
                                       -tf.ones(input_shape//2,dtype=dtype)],
                                    axis=0)
        else: # odd input shape
            init_weights = tf.concat( [tf.ones(input_shape//2, dtype=dtype),
                                      tf.constant([0.0], dtype=dtype),
                                      -tf.ones(input_shape//2, dtype=dtype)],
                                    axis=0)
        return tf.expand_dims(init_weights,axis=1)
    
class symmetricPenultimate_Init(tf.keras.initializers.Initializer):
    def __init__(self, mean, stddev):
        self.mean = mean
        self.stddev = stddev
    
    def __call__(self, shape, dtype=tf.float32, **kwargs):
        output_shape = shape[1]
        first_half_rows = tf.random.normal([shape[0],output_shape//2],
                                mean=self.mean, stddev=self.stddev, dtype=dtype)
        if output_shape % 2 == 0: # even output shape
            init_weights = tf.concat( [first_half_rows,
                                       first_half_rows],
                                    axis=1)
        else: # odd output shape
            init_weights = tf.concat( [first_half_rows,
                                        tf.ones([shape[0],1], dtype=dtype),
                                       first_half_rows],
                                    axis=1)
        return init_weights


In [ ]:
# actual neural net model
class Logit_NN(keras.layers.Layer):
    def __init__(self, hidden_dim=768, L = 10, activation='relu',
                     init_method='random',model_type='logit'):
        super(Logit_NN, self).__init__()
        self.hidden_dim = hidden_dim # hidden_dim
        self.L = L # number of layers
        self.init_method = init_method # random, zeros, or symmetric
        self.model_type = model_type
        self.layers = [layers.Dense(self.hidden_dim,
                            activation=activation) for j in range(L-2)]
        penultimate_init = 'glorot_uniform'
        if self.init_method == 'random':
            last_init = 'glorot_uniform'
        elif self.init_method == 'zeros':
            last_init = tf.keras.initializers.Zeros()
        else: # 'symmetric'
            penultimate_init = symmetricPenultimate_Init(mean=0.0,
                                            stddev=tf.math.sqrt(1/self.hidden_dim))
            last_init = symmetricLastLayer_Init()
        self.penultimate_init = penultimate_init
        self.last_init = last_init
        self.penultimate_layer = layers.Dense(self.hidden_dim/2,
                                activation=activation,
                                kernel_initializer=penultimate_init)
        if self.model_type == 'logit':
            self.last_layer = layers.Dense(1, activation='sigmoid',
                                kernel_initializer=last_init)
        else:
            self.last_layer = layers.Dense(1, activation='linear',
                                kernel_initializer=last_init)

    def call(self, x, training=False):
        # these will help calulate delta_j
        outputs = [x]
        intermediate_hadamard_prods = []
        for layer in self.layers:
            x = layer(x, training=training)
            outputs.append(x)
            hadamard_out = tf.linalg.diag(tf.cast(x > 0, dtype=tf.float32))
            intermediate_hadamard_prods.append(hadamard_out)
        x = self.penultimate_layer(x, training=training)
        outputs.append(x)
        hadamard_out = tf.linalg.diag(tf.cast(x > 0, dtype=tf.float32))
        intermediate_hadamard_prods.append(hadamard_out)
        x = self.last_layer(x, training=training)
        if self.init_method == 'symmetric' and self.model_type != 'logit':
            x = x+(0.5)
        return x, outputs, intermediate_hadamard_prods


# Training class with plotting

Here, we create a training object that will be used to 1) train the neural net object above, 2) plot the procession of the neural net output as training ensues, and 3) apply a permutation test and plot the histogram for it.  We will apply this training class to the different model situations above.

In [ ]:
class Trainer(object):
    def __init__(self, hidden_dim=50, L = 5, activation='relu',
                    init_method='random',model_type='logit', lr = 0.001,
                trainN=5000, testN=1000, d=20, batch_sz=32):
        # first set model that will be trained
        self.model_type = model_type
        self.lr = lr
        self.hidden_dim = hidden_dim
        self.L = L
        self.init_method = init_method
        self.model_type = model_type
        self.data_constructor = Data_Constructor(trainN=trainN, testN=testN,
                                                d=d, batch_sz=batch_sz)
        self.loss_fcn = tf.keras.losses.MeanSquaredError()
        if self.model_type == 'logit':
            self.loss_fcn = tf.keras.losses.BinaryCrossentropy()
        self.optimizer = keras.optimizers.SGD(learning_rate=self.lr)
        
    def train_and_plot(self, num_epochs, num_tests, m_perm=1000, n_bins=50):
        T_H0_mod1 = {j: [] for j in range(num_epochs)}
        T_mod1 = {j: [] for j in range(num_epochs)}
        T_H0_NTK1 = {j:[] for j in range(num_epochs)}
        T_NTK1 = {j:[] for j in range(num_epochs)}
        T_H0_mod2 = {j: [] for j in range(num_epochs)}
        T_mod2 = {j: [] for j in range(num_epochs)}
        T_H0_NTK2 = {j: [] for j in range(num_epochs)}
        T_NTK2 = {j : [] for j in range(num_epochs)}
        
        for test in range(num_tests):
            t1 = time.time()
            # create model 1 for testing on Gaussian mixture data
            # create model 2 for testing on regular Gaussian data
            # form NTK for both model 1 and model 2 and their respective data
            dset_train, dset_test, dset_train_prime, dset_test_prime = self.data_constructor.constructData()
            model_1 = Logit_NN(hidden_dim=self.hidden_dim, L=self.L,
                              init_method=self.init_method,model_type=self.model_type)
            model_2 = Logit_NN(hidden_dim=self.hidden_dim, L=self.L,
                              init_method=self.init_method,model_type=self.model_type)
            #print('Running test', test, 'for model 1.....')
            #T_H0_epoch_num_1, T_epoch_num_1 = self.train_model(model_1, dset_train,
            #                                        dset_test, num_epochs=num_epochs,
            #                                        m_perm=m_perm)
            #print('Running test', test, 'for model 2....')
            #T_H0_epoch_num_2, T_epoch_num_2 = self.train_model(model_2, dset_train_prime,
            #                                        dset_test_prime, num_epochs=num_epochs,
            #                                        m_perm=m_perm)
            if self.init_method != 'zeros':
                print('Running test', test, 'for NTK1....')
                T_H0_ntk1_epoch_num, T_ntk1_epoch_num = self.train_NTK(model_1,
                                                            num_epochs=num_epochs, m_perm=m_perm,
                                                            mod_type=1)
                print('Running test', test, 'for NTK2....')
                T_H0_ntk2_epoch_num, T_ntk2_epoch_num = self.train_NTK(model_2,
                                                            num_epochs=num_epochs, m_perm=m_perm,
                                                            mod_type = 2)
            for j in range(num_epochs):
                #T_H0_mod1[j].extend(T_H0_epoch_num_1[j])
                #T_mod1[j].append(T_epoch_num_1[j])
                #T_H0_mod2[j].extend(T_H0_epoch_num_2[j])
                #T_mod2[j].append(T_epoch_num_2[j])
                if self.init_method != 'zeros':
                    T_H0_NTK1[j].extend(T_H0_ntk1_epoch_num[j])
                    T_NTK1[j].append(T_ntk1_epoch_num[j])
                    T_H0_NTK2[j].extend(T_H0_ntk2_epoch_num[j])
                    T_NTK2[j].append(T_ntk2_epoch_num[j])
            del_T = time.time() - t1
            print('Took', del_T, 'seconds to finish one test...')
            print('Estimated hours to finish all tests:', del_T*(num_tests - test)/3600)
            if test % 2 == 0:
                clear_output(wait=True)
        self.T_2_H0 = {'model on mixture data':[T_H0_mod1, T_mod1],
                  'model on Gaussian data':[T_H0_mod2, T_mod2],
                  'NTK on mixture data':[T_H0_NTK1, T_NTK1],
                  'NTK on Gaussian data':[T_H0_NTK2, T_NTK2]}
        for j in range(num_epochs):
            for mod_key in self.T_2_H0.keys():
                if (not self.init_method == 'zeros') and ('NTK' in mod_key):
                    T_H0 = self.T_2_H0[mod_key][0]
                    T = self.T_2_H0[mod_key][1]
                    plt.hist(tf.concat(T_H0[j],axis=0).numpy(), bins=50,
                             label='permutation tests',density=True)
                    y_vals = tf.zeros(tf.concat(T[j],axis=0).shape,dtype=tf.float32)
                    #plt.plot(tf.concat(T[j],axis=0).numpy(), y_vals ,color='r',marker='o',markersize=15)
                    plt.hist(tf.concat(T[j],axis=0).numpy(), bins=3,label='actual',
                                        density=True)
                    plt.legend(loc='upper left')
                    title_name = 'Permutation Test of '+mod_key+' for Epoch'+str(j)
                    plt.title(title_name)
                    plt.show()
            
    def train_model(self, model, train_set, test_set, num_epochs=10, m_perm=1000, plot_data=False):
        T_H0_epoch_num = {j: [] for j in range(num_epochs)}
        T_epoch_num = {j:0 for j in range(num_epochs)}
        for j, epoch in enumerate(range(num_epochs)):
            for k, data in enumerate(train_set):
                features, labels = data
                with tf.GradientTape() as tape:
                    prediction, _, _ = model(features,training=True)
                    loss = self.loss_fcn(prediction, labels)
                grads = tape.gradient(loss,model.trainable_variables)
                self.optimizer.apply_gradients(list(zip(grads,
                                            model.trainable_variables)))
            feats_2_plot = []
            predictions_2_plot = []
            labels_2_plot = []
            for k, data in enumerate(test_set):
                # aggregate information for plotting
                features, labels = data
                feats_2_plot.append(features[:,:2])
                labels_2_plot.append(labels)
                prediction, _, _ = model(features, training=False)
                predictions_2_plot.append(prediction)
            # plot
            plotting_feats = tf.concat(feats_2_plot, axis=0)
            plotting_preds = tf.concat(predictions_2_plot, axis=0)
            plotting_labels = tf.concat(labels_2_plot, axis=0)
            if plot_data:
                plt.scatter(plotting_feats.numpy()[:,0],
                            plotting_feats.numpy()[:,1],
                            c=plotting_preds.numpy())
                plt.colorbar()
                plt.title('Actual Predictions of Model')
                plt.show()
                # plotting for comparison to labels
                plt.scatter(plotting_feats.numpy()[:,0],
                            plotting_feats.numpy()[:,1],
                            c=plotting_labels.numpy())
                plt.colorbar()
                plt.title('Labels for Dataset')
                plt.show()
            # plot permutation test
            T_1 = tf.math.reduce_mean(tf.gather(plotting_preds,
                                  tf.concat(tf.where(plotting_labels==1),axis=0)[:,0], axis=0))
            T_0 = tf.math.reduce_mean(tf.gather(plotting_preds,
                                  tf.concat(tf.where(plotting_labels==0),axis=0)[:,0], axis=0))
            T = T_1 - T_0
            # apply permutation test
            T_H0 = []
            for k in range(m_perm):
                dummy_labels = tf.random.shuffle(plotting_labels)
                T_1_dummy = tf.math.reduce_mean(tf.gather(plotting_preds,
                                  tf.concat(tf.where(dummy_labels==1),axis=0)[:,0], axis=0))
                T_0_dummy = tf.math.reduce_mean(tf.gather(plotting_preds,
                                  tf.concat(tf.where(dummy_labels==0),axis=0)[:,0], axis=0))
                T_H0.append(T_1_dummy - T_0_dummy)
            T_H0_epoch_num[j].extend(T_H0)
            T_epoch_num[j] += T
        return T_H0_epoch_num, T_epoch_num
            
    def calc_reps(features, model):
        # calculating the delta_j's and outputs
        x, outputs, hadamard_outs = model(features)
        delta_j = model.last_layer.weights[0]
        deltas = [tf.squeeze(delta_j)]
        delta_j = tf.matmul( model.penultimate_layer.weights[0],
                               tf.matmul(hadamard_outs[-1], delta_j))
        deltas.append(tf.squeeze(delta_j))
        for j in range(model.L-3, -1, -1):
            delta_j = tf.matmul(model.layers[j].weights[0],
                                tf.matmul(hadamard_outs[j], delta_j))
            deltas.append(tf.squeeze(delta_j))
        deltas.reverse()
        return x, outputs, deltas


    def concat_reps(train_set, model):
        outs = []
        delta_data = []
        for j, data in enumerate(train_set):
            feats, label = data
            x, outputs, deltas = Trainer.calc_reps(feats, model)
            if len(outs) == 0:
                outs = outputs
                delta_data = deltas
            else:
                 for j in range(len(outs)):
                        if j != len(outs) - 1:
                            delta_data[j] = tf.concat([delta_data[j],deltas[j]],axis=0)
                        outs[j] = tf.concat([outs[j],outputs[j]],axis=0)
        return outs, delta_data

    def form_NTK(outputs, deltas, nn_type='logit'):
        ntk = 0
        for k in range(len(outputs)):
            if k == len(outputs)-1:
                d_j = tf.matmul(tf.expand_dims(deltas[k],
                                axis=0),
                                tf.transpose(tf.expand_dims(deltas[k],
                                                            axis=0)))
            else:
                d_j = tf.matmul(deltas[k],tf.transpose(deltas[k]))
            x_j = tf.matmul(outputs[k], tf.transpose(outputs[k]))
            ntk = ntk + d_j*x_j
        if nn_type == 'logit':
            ntk = (1/4)*ntk
        # normalize ntk
        k_hat = tf.linalg.tensor_diag_part(ntk)
        k_hat = tf.expand_dims(1/tf.math.sqrt(tf.linalg.tensor_diag_part(ntk)),axis=1)
        k_hat_prime = tf.linalg.matmul(k_hat, tf.transpose(k_hat))
        ntk = ntk*k_hat_prime
        return ntk
    
    def train_NTK(self,model, num_epochs, m_perm=1000, mod_type = 1):
        dset_train, dset_test, dset_train_prime, dset_test_prime = self.data_constructor.constructData()
        if mod_type == 2:
            dset_train = dset_train_prime
            dset_test = dset_test_prime
        out, delta = Trainer.concat_reps(dset_train.concatenate(dset_test),model)
        ntk = Trainer.form_NTK(out, delta, nn_type=self.model_type)
        train_N = self.data_constructor.trainN*2
        init = tf.random.normal([train_N , 1])
        init = tf.zeros([train_N, 1])
        # split up NTK into different components
        ntk_train = ntk[:train_N, :train_N] # rows and columns from train
        ntk_test = ntk[train_N:, :train_N] # rows from test and columns from train
        T_H0_epoch_num = {j:[] for j in range(num_epochs)}
        T_epoch_num = {j: 0 for j in range(num_epochs)}
        D, V = np.linalg.eigh(ntk_train)
        for j in range(num_epochs):
            for k, batch in enumerate(dset_train):
                feats, label = batch
                batch_sz = feats.shape[0]
                if self.model_type == 'logit':
                    dy = label - tf.math.sigmoid(init[j*batch_sz:(j+1)*batch_sz])
                else:
                    dy = label - init[j*batch_sz: (j+1)*batch_sz]
                gradstep = tf.linalg.matmul(tf.transpose(tf.gather(ntk_train,
                                        list(range(j*batch_sz,
                                        (j+1)*batch_sz)), axis=0 )), dy)
                #print('gradstep:', gradstep)
                # (1e-5)*
                init = init - self.lr*gradstep
            feats_2_plot = []
            predictions_2_plot = []
            labels_2_plot = []
            for k, batch in enumerate(dset_test):
                feats, label = batch
                batch_sz = feats.shape[0]
                labels_2_plot.append(label)
            plotting_labels = tf.concat(labels_2_plot, axis=0)
            coefs = tf.linalg.matmul(V,tf.linalg.matmul(tf.linalg.diag(1/D),
                                                tf.linalg.matmul(V.T, init)))
            plotting_preds = tf.matmul(ntk_test, coefs)
            T_1 = tf.math.reduce_mean(tf.gather(plotting_preds,
                                    tf.concat(tf.where(plotting_labels==1),
                                             axis=0)[:,0], axis=0))
            T_0 = tf.math.reduce_mean(tf.gather(plotting_preds,
                                    tf.concat(tf.where(plotting_labels==0),
                                             axis=0)[:,0], axis=0))
            T = T_1 - T_0
            # apply permutation test
            T_H0 = []
            for k in range(m_perm):
                dummy_labels = tf.random.shuffle(plotting_labels)
                T_1_dummy = tf.math.reduce_mean(tf.gather(plotting_preds,
                                  tf.concat(tf.where(dummy_labels==1),axis=0)[:,0], axis=0))
                T_0_dummy = tf.math.reduce_mean(tf.gather(plotting_preds,
                                  tf.concat(tf.where(dummy_labels==0),axis=0)[:,0], axis=0))
                T_H0.append(T_1_dummy - T_0_dummy)
            T_H0_epoch_num[j].extend(T_H0)
            T_epoch_num[j] += T
        del dset_train, 
        return T_H0_epoch_num, T_epoch_num

In [ ]:
# hyperparams for series of tests
num_epochs = 10
L = 2
activation='relu'
lr = 0.001
num_tests = 10
m_perm = 1000

# Tests with Hidden Dim 4000 ( same as number of training points)

We run tests the different initialization methods for each of logit vs. l2_loss cases.

In [ ]:
hidden_dim = 4000
# 50, 4000, 80000

## Symmetric Initialization
### Logit

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='symmetric',model_type='logit', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

### L2-Loss

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='symmetric',model_type='l2', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

## Random Initialization
### Logit

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='random',model_type='logit', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

### L2-Loss

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='random',model_type='l2', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

## Zero Initialization
### Logit

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='zeros',model_type='logit', lr = 1e-2*lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

### L2-Loss

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='zeros',model_type='l2', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

# Tests with hidden dim 80000

We run tests the different initialization methods for each of logit vs. l2_loss cases with hidden dimension now 80000.

In [ ]:
hidden_dim = 40000
num_tests = 10

## Symmetric Initialization
### Logit

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='symmetric',model_type='logit', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

### L2-Loss

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='symmetric',model_type='l2', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

## Random Initialization
### Logit

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='random',model_type='logit', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

### L2-Loss

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='random',model_type='l2', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)

## Zero Initialization
### Logit

In [ ]:
trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='zeros',model_type='logit', lr = 1e-2*lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)


### L2-Loss

In [ ]:

trainer_obj = Trainer(hidden_dim=hidden_dim, L = L, activation=activation,
            init_method='zeros',model_type='l2', lr = lr,
                trainN=2000, testN=500, d=20, batch_sz=32)
trainer_obj.train_and_plot(num_epochs=num_epochs, 
                           num_tests=num_tests, m_perm=m_perm)